<a href="https://colab.research.google.com/github/Locke0/Text-Summarizer/blob/main/Text_Summarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Imports


In [1]:
#imports

from nltk.cluster.util import cosine_distance
from nltk.tokenize import word_tokenize

import numpy as np
import networkx as nx
# loading data
import requests


#Helpful Links

https://www.analyticsvidhya.com/blog/2019/06/comprehensive-guide-text-summarization-using-deep-learning-python/

https://towardsdatascience.com/understand-text-summarization-and-create-your-own-summarizer-in-python-b26a9f09fc70

# Data Processing


## Loading and Cleaning Data
https://towardsdatascience.com/nlp-building-text-cleanup-and-preprocessing-pipeline-eba4095245a0


In [2]:
# text: https://www.gutenberg.org/cache/epub/21077/pg21077.txt
# request tutorial: https://www.youtube.com/watch?v=mKP7ChzJBqg

# loading books

result = requests.get('https://www.gutenberg.org/cache/epub/21077/pg21077.txt')
 
result.encoding = result.apparent_encoding

text = result.text

text

'The Project Gutenberg eBook, The Mind and the Brain, by Alfred Binet\r\n\r\n\r\nThis eBook is for the use of anyone anywhere at no cost and with\r\nalmost no restrictions whatsoever.  You may copy it, give it away or\r\nre-use it under the terms of the Project Gutenberg License included\r\nwith this eBook or online at www.gutenberg.org\r\n\r\n\r\n\r\n\r\n\r\nTitle: The Mind and the Brain\r\n       Being the Authorised Translation of L\'Âme et le Corps\r\n\r\n\r\nAuthor: Alfred Binet\r\n\r\n\r\n\r\nRelease Date: April 14, 2007  [eBook #21077]\r\n\r\nLanguage: English\r\n\r\n\r\n***START OF THE PROJECT GUTENBERG EBOOK THE MIND AND THE BRAIN***\r\n\r\n\r\nE-text prepared by Curtis Weyant, Sankar Viswanathan, and the Project\r\nGutenberg Online Distributed Proofreading Team (http://www.pgdp.net) from\r\npage images generously made available by Digital Case, the Kelvin Smith\r\nLibrary, Case Western Reserve University\r\n(http://library.case.edu/digitalcase/)\r\n\r\n\r\n\r\nNote: Images of

##Testing Text

In [3]:
text = '''The Project Gutenberg eBook, The Mind and the Brain, by Alfred Binet


This eBook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  You may copy it, give it away or
re-use it under the terms of the Project Gutenberg License included
with this eBook or online at www.gutenberg.org





Title: The Mind and the Brain
       Being the Authorised Translation of L'Âme et le Corps


Author: Alfred Binet



Release Date: April 14, 2007  [eBook #21077]

Language: English


***START OF THE PROJECT GUTENBERG EBOOK THE MIND AND THE BRAIN***


E-text prepared by Curtis Weyant, Sankar Viswanathan, and the Project
Gutenberg Online Distributed Proofreading Team (http://www.pgdp.net) from
page images generously made available by Digital Case, the Kelvin Smith
Library, Case Western Reserve University
(http://library.case.edu/digitalcase/)



Note: Images of the original pages are available through
      Digital Case, the Kelvin Smith Library, Case Western
      Reserve University. See
      http://digitalcase.case.edu:9000/fedora/get/ksl:binmin00/binmin00.pdf


Transcriber's Note:

   References are made to footnotes in other footnotes and index.
   The footnotes are serially numbered and placed at the end of
   each chapter. Consequently the references in the footnotes
   and index have been corrected to indicate the footnote number.




International Scientific Series.

Volume LXXXIX.

(The International Scientific Series)

Edited by F. Legge

THE MIND AND THE BRAIN

by

ALFRED BINET

Directeur du Laboratoire de Psychologie
à la Sorbonne

Being the Authorised Translation of

_L'Âme et le Corps_







London
Kegan Paul, Trench, Trübner & Co. Ltd
Dryden House, Gerrard Street, W.
1907

'''

## Remove HTML Tags

In [4]:
from bs4 import BeautifulSoup

def remove_html_tags(text):
  return BeautifulSoup(text, 'html.parser').get_text()

clean_text = remove_html_tags(text)
clean_text

"The Project Gutenberg eBook, The Mind and the Brain, by Alfred Binet\n\n\nThis eBook is for the use of anyone anywhere at no cost and with\nalmost no restrictions whatsoever.  You may copy it, give it away or\nre-use it under the terms of the Project Gutenberg License included\nwith this eBook or online at www.gutenberg.org\n\n\n\n\n\nTitle: The Mind and the Brain\n       Being the Authorised Translation of L'Âme et le Corps\n\n\nAuthor: Alfred Binet\n\n\n\nRelease Date: April 14, 2007  [eBook #21077]\n\nLanguage: English\n\n\n***START OF THE PROJECT GUTENBERG EBOOK THE MIND AND THE BRAIN***\n\n\nE-text prepared by Curtis Weyant, Sankar Viswanathan, and the Project\nGutenberg Online Distributed Proofreading Team (http://www.pgdp.net) from\npage images generously made available by Digital Case, the Kelvin Smith\nLibrary, Case Western Reserve University\n(http://library.case.edu/digitalcase/)\n\n\n\nNote: Images of the original pages are available through\n      Digital Case, the Kelvin

##Removing Accented Characters

In [5]:
# imports
import unicodedata

# function to remove accented characters
def remove_accented_chars(text):
    new_text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return new_text
# call function
clean_text = remove_accented_chars(clean_text)
clean_text

"The Project Gutenberg eBook, The Mind and the Brain, by Alfred Binet\n\n\nThis eBook is for the use of anyone anywhere at no cost and with\nalmost no restrictions whatsoever.  You may copy it, give it away or\nre-use it under the terms of the Project Gutenberg License included\nwith this eBook or online at www.gutenberg.org\n\n\n\n\n\nTitle: The Mind and the Brain\n       Being the Authorised Translation of L'Ame et le Corps\n\n\nAuthor: Alfred Binet\n\n\n\nRelease Date: April 14, 2007  [eBook #21077]\n\nLanguage: English\n\n\n***START OF THE PROJECT GUTENBERG EBOOK THE MIND AND THE BRAIN***\n\n\nE-text prepared by Curtis Weyant, Sankar Viswanathan, and the Project\nGutenberg Online Distributed Proofreading Team (http://www.pgdp.net) from\npage images generously made available by Digital Case, the Kelvin Smith\nLibrary, Case Western Reserve University\n(http://library.case.edu/digitalcase/)\n\n\n\nNote: Images of the original pages are available through\n      Digital Case, the Kelvin

##Expanding Contractions

In [6]:
# imports
!pip install contractions
import contractions

clean_text = contractions.fix(clean_text)
clean_text

     |████████████████████████████████| 283 kB 8.4 MB/s 
     |████████████████████████████████| 321 kB 46.2 MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85454 sha256=45def0ac4ddbda2e019c3797c3ef5aa5472805fd9179283c4f0e9c1049b764d6
  Stored in directory: /root/.cache/pip/wheels/25/19/a6/8f363d9939162782bb8439d886469756271abc01f76fbd790f
Successfully built pyahocorasick


"The Project Gutenberg eBook, The Mind and the Brain, by Alfred Binet\n\n\nThis eBook is for the use of anyone anywhere at no cost and with\nalmost no restrictions whatsoever.  You may copy it, give it away or\nre-use it under the terms of the Project Gutenberg License included\nwith this eBook or online at www.gutenberg.org\n\n\n\n\n\nTitle: The Mind and the Brain\n       Being the Authorised Translation of L'Ame et le Corps\n\n\nAuthor: Alfred Binet\n\n\n\nRelease Date: April 14, 2007  [eBook #21077]\n\nLanguage: English\n\n\n***START OF THE PROJECT GUTENBERG EBOOK THE MIND AND THE BRAIN***\n\n\nE-text prepared by Curtis Weyant, Sankar Viswanathan, and the Project\nGutenberg Online Distributed Proofreading Team (http://www.pgdp.net) from\npage images generously made available by Digital Case, the Kelvin Smith\nLibrary, Case Western Reserve University\n(http://library.case.edu/digitalcase/)\n\n\n\nNote: Images of the original pages are available through\n      Digital Case, the Kelvin

##Removing Special Characters

In [7]:
# imports
import re
# function to remove special characters
def remove_special_characters(text):
    # define the pattern to keep
    pat = r'[^a-zA-z0-9.,!?/:;\"\'\s]' 
    return re.sub(pat, '', text)
 
# call function
clean_text = remove_special_characters(clean_text)
clean_text

"The Project Gutenberg eBook, The Mind and the Brain, by Alfred Binet\n\n\nThis eBook is for the use of anyone anywhere at no cost and with\nalmost no restrictions whatsoever.  You may copy it, give it away or\nreuse it under the terms of the Project Gutenberg License included\nwith this eBook or online at www.gutenberg.org\n\n\n\n\n\nTitle: The Mind and the Brain\n       Being the Authorised Translation of L'Ame et le Corps\n\n\nAuthor: Alfred Binet\n\n\n\nRelease Date: April 14, 2007  [eBook 21077]\n\nLanguage: English\n\n\nSTART OF THE PROJECT GUTENBERG EBOOK THE MIND AND THE BRAIN\n\n\nEtext prepared by Curtis Weyant, Sankar Viswanathan, and the Project\nGutenberg Online Distributed Proofreading Team http://www.pgdp.net from\npage images generously made available by Digital Case, the Kelvin Smith\nLibrary, Case Western Reserve University\nhttp://library.case.edu/digitalcase/\n\n\n\nNote: Images of the original pages are available through\n      Digital Case, the Kelvin Smith Librar

##Removing Numbers

In [8]:
# function to remove numbers
def remove_numbers(text):
    # define the pattern to keep
    pattern = r'[^a-zA-z.,!?/:;\"\'\s]' 
    return re.sub(pattern, '', text)
 
# call function
clean_text = remove_numbers(clean_text)
clean_text

"The Project Gutenberg eBook, The Mind and the Brain, by Alfred Binet\n\n\nThis eBook is for the use of anyone anywhere at no cost and with\nalmost no restrictions whatsoever.  You may copy it, give it away or\nreuse it under the terms of the Project Gutenberg License included\nwith this eBook or online at www.gutenberg.org\n\n\n\n\n\nTitle: The Mind and the Brain\n       Being the Authorised Translation of L'Ame et le Corps\n\n\nAuthor: Alfred Binet\n\n\n\nRelease Date: April ,   [eBook ]\n\nLanguage: English\n\n\nSTART OF THE PROJECT GUTENBERG EBOOK THE MIND AND THE BRAIN\n\n\nEtext prepared by Curtis Weyant, Sankar Viswanathan, and the Project\nGutenberg Online Distributed Proofreading Team http://www.pgdp.net from\npage images generously made available by Digital Case, the Kelvin Smith\nLibrary, Case Western Reserve University\nhttp://library.case.edu/digitalcase/\n\n\n\nNote: Images of the original pages are available through\n      Digital Case, the Kelvin Smith Library, Case Wes

##Removing extra whitespaces and tabs

In [9]:
# function to remove special characters
def remove_extra_whitespace_tabs(text):
    #pattern = r'^\s+$|\s+$'
    pattern = r'^\s*|\s\s*'
    return re.sub(pattern, ' ', text).strip()
# call function
clean_text = remove_extra_whitespace_tabs(clean_text)
clean_text

"The Project Gutenberg eBook, The Mind and the Brain, by Alfred Binet This eBook is for the use of anyone anywhere at no cost and with almost no restrictions whatsoever. You may copy it, give it away or reuse it under the terms of the Project Gutenberg License included with this eBook or online at www.gutenberg.org Title: The Mind and the Brain Being the Authorised Translation of L'Ame et le Corps Author: Alfred Binet Release Date: April , [eBook ] Language: English START OF THE PROJECT GUTENBERG EBOOK THE MIND AND THE BRAIN Etext prepared by Curtis Weyant, Sankar Viswanathan, and the Project Gutenberg Online Distributed Proofreading Team http://www.pgdp.net from page images generously made available by Digital Case, the Kelvin Smith Library, Case Western Reserve University http://library.case.edu/digitalcase/ Note: Images of the original pages are available through Digital Case, the Kelvin Smith Library, Case Western Reserve University. See http://digitalcase.case.edu:/fedora/get/ksl:

##Stemming (Not Used in This Project)

In [10]:
# # imports
# from nltk.stem import PorterStemmer
# # function for stemming
# def get_stem(text):
#     stemmer = nltk.porter.PorterStemmer()
#     text = ' '.join([stemmer.stem(word) for word in text.split()])
#     return text
# # call function
# clean_text = get_stem(clean_text)

## Lemmatization (More Accurate Than Stemming)

In [11]:
# imports
import spacy
nlp = spacy.load('en',parse=True,tag=True, entity=True)
# function to remove special characters
def get_lem(text):
    text = nlp(text)
    text = ' '.join([word.lemma_ if word.lemma_ != '-PRON-' else word.text for word in text])
    return text
# call function
get_lem(clean_text)

"the Project Gutenberg eBook , the Mind and the Brain , by Alfred Binet this eBook be for the use of anyone anywhere at no cost and with almost no restriction whatsoever . You may copy it , give it away or reuse it under the term of the Project Gutenberg License include with this ebook or online at www.gutenberg.org title : the Mind and the Brain be the authorised Translation of L'Ame et le Corps author : Alfred Binet Release date : April , [ ebook ] language : english start of the PROJECT GUTENBERG EBOOK the mind and the BRAIN Etext prepare by Curtis Weyant , Sankar Viswanathan , and the Project Gutenberg Online distribute Proofreading Team http://www.pgdp.net from page image generously make available by Digital Case , the Kelvin Smith Library , Case Western Reserve University http://library.case.edu/digitalcase/ note : image of the original page be available through Digital Case , the Kelvin Smith Library , Case Western Reserve University . see http://digitalcase.case.edu:/fedora / g

##Removing Stopwords

In [14]:
# imports
import nltk
from nltk.tokenize import ToktokTokenizer
nltk.download('stopwords')

tokenizer = ToktokTokenizer()
stopword_list = nltk.corpus.stopwords.words('english')
# custom: removing words from list
stopword_list.remove('not')
# function to remove stopwords
def remove_stopwords(text):
    # convert sentence into token of words
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    # check in lowercase 
    t = [token for token in tokens if token.lower() not in stopword_list]
    text = ' '.join(t)    
    return text
# call function
remove_stopwords(clean_text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


"Project Gutenberg eBook , Mind Brain , Alfred Binet eBook use anyone anywhere cost almost restrictions whatsoever. may copy , give away reuse terms Project Gutenberg License included eBook online www.gutenberg.org Title : Mind Brain Authorised Translation L ' Ame et le Corps Author : Alfred Binet Release Date : April , [ eBook ] Language : English START PROJECT GUTENBERG EBOOK MIND BRAIN Etext prepared Curtis Weyant , Sankar Viswanathan , Project Gutenberg Online Distributed Proofreading Team http://www.pgdp.net page images generously made available Digital Case , Kelvin Smith Library , Case Western Reserve University http://library.case.edu/digitalcase/ Note : Images original pages available Digital Case , Kelvin Smith Library , Case Western Reserve University. See http://digitalcase.case.edu : / fedora/get/ksl : binmin/binmin.pdf Transcriber ' Note : References made footnotes footnotes index. footnotes serially numbered placed end chapter. Consequently references footnotes index cor

##Lower Case



In [15]:
clean_text.lower()

"the project gutenberg ebook, the mind and the brain, by alfred binet this ebook is for the use of anyone anywhere at no cost and with almost no restrictions whatsoever. you may copy it, give it away or reuse it under the terms of the project gutenberg license included with this ebook or online at www.gutenberg.org title: the mind and the brain being the authorised translation of l'ame et le corps author: alfred binet release date: april , [ebook ] language: english start of the project gutenberg ebook the mind and the brain etext prepared by curtis weyant, sankar viswanathan, and the project gutenberg online distributed proofreading team http://www.pgdp.net from page images generously made available by digital case, the kelvin smith library, case western reserve university http://library.case.edu/digitalcase/ note: images of the original pages are available through digital case, the kelvin smith library, case western reserve university. see http://digitalcase.case.edu:/fedora/get/ksl:

#Generating sentences

In [16]:
import string

def read_article(text):
  # text = re.sub(r'[^a-zA-z0-9.,!?/:;\"\'\s]' , ' ', text)
  # article = clean_text.split("\. |\.[a-zA-Z]")
  article = text.split('. ')
  sentences = []
  # result = []

  for sentence in article:
    #deleting any character that IS NOT a-z OR A-Z
    # sentences.append(sentence.replace("--|[^0-9a-zA-Z]+", " ").split(" "))
    if sentence.startswith('http') == False and sentence.startswith('www') == False:
      sentence = ''.join([c for c in sentence if c not in string.punctuation])
    sentences.append(sentence.split())



  return sentences
  
clean_text = read_article(clean_text)
clean_text

[['The',
  'Project',
  'Gutenberg',
  'eBook',
  'The',
  'Mind',
  'and',
  'the',
  'Brain',
  'by',
  'Alfred',
  'Binet',
  'This',
  'eBook',
  'is',
  'for',
  'the',
  'use',
  'of',
  'anyone',
  'anywhere',
  'at',
  'no',
  'cost',
  'and',
  'with',
  'almost',
  'no',
  'restrictions',
  'whatsoever'],
 ['You',
  'may',
  'copy',
  'it',
  'give',
  'it',
  'away',
  'or',
  'reuse',
  'it',
  'under',
  'the',
  'terms',
  'of',
  'the',
  'Project',
  'Gutenberg',
  'License',
  'included',
  'with',
  'this',
  'eBook',
  'or',
  'online',
  'at',
  'wwwgutenbergorg',
  'Title',
  'The',
  'Mind',
  'and',
  'the',
  'Brain',
  'Being',
  'the',
  'Authorised',
  'Translation',
  'of',
  'LAme',
  'et',
  'le',
  'Corps',
  'Author',
  'Alfred',
  'Binet',
  'Release',
  'Date',
  'April',
  'eBook',
  'Language',
  'English',
  'START',
  'OF',
  'THE',
  'PROJECT',
  'GUTENBERG',
  'EBOOK',
  'THE',
  'MIND',
  'AND',
  'THE',
  'BRAIN',
  'Etext',
  'prepared',
  'by

##Similarity matrix

In [93]:
# https://www.geeksforgeeks.org/python-measure-similarity-between-two-sentences-using-cosine-similarity/
# https://towardsdatascience.com/understand-text-summarization-and-create-your-own-summarizer-in-python-b26a9f09fc70


def build_similarity_matrix(sentences, stop_words):
  # Create an empty similarity matrix
  similarity_matrix = np.zeros((len(sentences), len(sentences)))

  for idx1 in range(len(sentences)):
       for idx2 in range(len(sentences)):
          if idx1 == idx2: #ignore if both are same sentences
              continue 
          similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stop_words)
  return similarity_matrix

##Generate Summary Method

In [94]:
def generate_summary(text, top_n=5):
  stop_words = stopwords.words('english')
  summarize_text = []
  # Step 1 - Read text and tokenize
  sentences =  read_article(text)

  # Step 2 - Generate Similary Martix across sentences
  sentence_similarity_martix = build_similarity_matrix(sentences, stop_words)

  # Step 2 - Generate Similary Martix across sentences
  sentence_similarity_martix = build_similarity_matrix(sentences, stop_words)

  # Step 3 - Rank sentences in similarity martix
  sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_martix)
  scores = nx.pagerank(sentence_similarity_graph)

  # Step 4 - Sort the rank and pick top sentences
  ranked_sentence = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)    
  print("Indexes of top ranked_sentence order are ", ranked_sentence)
  
  for i in range(top_n):
    print(i)
    print(ranked_sentence)
    summarize_text.append(" ".join(ranked_sentence[i][1]))
    
  # Step 5 - Offcourse, output the summarize texr
    print("Summarize Text: \n", ". ".join(summarize_text))



##Visualization

##Modeling

In [95]:
# clustering: https://medium.com/@rohithramesh1991/unsupervised-text-clustering-using-natural-language-processing-nlp-1a8bc18b048d\




##Summarizing


In [96]:
generate_summary(text, top_n=5)

NameError: ignored